## Data Outline:

In [20]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://media1-production-mightynetworks.imgix.net/asset/26517892/er_-_data-analyst-fc-barcelona-20-21-db.png?ixlib=rails-0.3.0&fm=jpg&q=75&auto=format")

Importing the database:

In [1]:
!pip install psycopg2

In [8]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

db_config = {'user': 'practicum_student',         # username             
                      'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password             
                      'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',             
                      'port': 6432,              # connection port             
                      'db':  "data-analyst-fc-barcelona-20-21-db"}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                                                             db_config['pwd'], 
                                                                                             db_config['host'], 
                                                                                             db_config['port'], 
                                                                                             db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Creating a pandas platform to see the results:

In [3]:
def Pandaquery(q):
    return pd.io.sql.read_sql(q, con = engine)

What is the position played by the majority of the squad?

In [15]:
query = '''
        SELECT
        position,
        COUNT(position) AS number_of_players
        FROM
        squad
        GROUP BY
        position
        ORDER BY
        number_of_players DESC
        LIMIT 1;
        '''
Pandaquery(query)

,position,number_of_players
0,df,9


Which players played the most games this season?

In [19]:
query = '''
        SELECT
        squad.first_name,
        squad.last_name,
        games_played.league + games_played.champions + games_played.cup + games_played.supercup AS total_games_played
        FROM
        squad
        INNER JOIN
        games_played ON squad.player_id = games_played.player_id
        ORDER BY
        total_games_played DESC
        ;
        '''
Pandaquery(query)

,first_name,last_name,total_games_played
0,Pedro,Lopez,52
1,Antoine,Griezmann,51
2,Frenkie,De Jong,51
3,Sergio,Busquets,50
4,Jordi,Alba,49
5,Lionel,Messi,48
6,Clément,Lenglet,48
7,Ousmane,Dembélé,44
8,Marc-André,Ter Stegen,42
9,Francisco,Trincão,42


Out of the players who played less than 1000 minutes in the league who scored the most goals?

In [20]:
query = '''
        SELECT
        squad.first_name,
        squad.last_name,
        league_stats.league_goals
        FROM
        league_stats
        INNER JOIN
        squad ON squad.player_id = league_stats.player_id
        WHERE
        league_stats.league_minutes < 1000
        ORDER BY
        league_stats.league_goals DESC
        ;
        '''
Pandaquery(query)

,first_name,last_name,league_goals
0,Ansu,Fati,4
1,Francisco,Trincão,3
2,Philippe,Coutinho,2
3,Ilaix,Moriba,1
4,Sergi,Roberto,1
5,Junior,Firpo,1
6,Riqui,Puig,1
7,Norberto,Murara,0
8,Álex,Collado,0
9,Iñaki,Peña,0


Which players played the most minutes in the league? Show top 5.

In [21]:
query = '''
        SELECT
        squad.first_name,
        squad.last_name,
        league_stats.league_minutes
        FROM
        league_stats
        INNER JOIN
        squad ON squad.player_id = league_stats.player_id
        ORDER BY
        league_stats.league_minutes DESC
        LIMIT 5
        ;
        '''
Pandaquery(query)

,first_name,last_name,league_minutes
0,Frenkie,De Jong,3158
1,Jordi,Alba,3030
2,Lionel,Messi,3022
3,Marc-André,Ter Stegen,2790
4,Antoine,Griezmann,2619


How many assists were made by the forwards?

In [23]:
query = '''
        SELECT
        squad.first_name,
        squad.last_name,
        league_stats.league_assists + champions_stats.champions_assists+ cup_stats.cup_assists + supercup_stats.supercup_assists AS total_assists   
        FROM
        squad
        INNER JOIN league_stats ON squad.player_id = league_stats.player_id
        INNER JOIN champions_stats ON champions_stats.player_id = squad.player_id
        INNER JOIN cup_stats ON cup_stats.player_id = squad.player_id
        INNER JOIN supercup_stats ON supercup_stats.player_id = squad.player_id
        WHERE
        squad.position = 'fw'
        ORDER BY
        total_assists DESC
        ;
        '''
Pandaquery(query)

,first_name,last_name,total_assists
0,Lionel,Messi,14
1,Antoine,Griezmann,13
2,Ousmane,Dembélé,5
3,Ansu,Fati,4
4,Martin,Braithwaite,4
5,Francisco,Trincão,2
6,Konard,De La Fuente,0
7,Álex,Collado,0


Who are the top 3 players with the best goals per minute record? 

In [33]:
query = '''
        SELECT
        squad.first_name,
        squad.last_name,
        (league_stats.league_minutes + champions_stats.champions_minutes + cup_stats.cup_minutes + supercup_stats.supercup_minutes)  / (league_stats.league_goals + champions_stats.champions_goals + cup_stats.cup_goals + supercup_stats.supercup_goals) AS goals_per_minute
        FROM
        squad
        INNER JOIN league_stats ON squad.player_id = league_stats.player_id
        INNER JOIN champions_stats ON champions_stats.player_id = squad.player_id
        INNER JOIN cup_stats ON cup_stats.player_id = squad.player_id
        INNER JOIN supercup_stats ON supercup_stats.player_id = squad.player_id
        WHERE
        league_stats.league_goals + champions_stats.champions_goals + cup_stats.cup_goals + supercup_stats.supercup_goals > 0
        ORDER BY
        goals_per_minute
        LIMIT 3
        ;
        '''
Pandaquery(query)

,first_name,last_name,goals_per_minute
0,Lionel,Messi,110
1,Ansu,Fati,119
2,Antoine,Griezmann,195


How many players did not contribute goals or assists in the league?

In [35]:
query = '''
        SELECT
        COUNT(player_id)
        FROM
        league_stats
        WHERE
        league_goals = 0 AND league_assists = 0
        ;
        '''
Pandaquery(query)

,count
0,10


Who is the player that has the most yellow cards in the league?

In [43]:
query = '''
        SELECT
        squad.first_name,
        squad.last_name,
        league_stats.league_minutes,
        league_stats.league_yellow
        FROM
        league_stats
        INNER JOIN
        squad ON squad.player_id = league_stats.player_id
        ORDER BY
        league_stats.league_yellow DESC,
        league_stats.league_minutes DESC
        LIMIT 1;
        '''
Pandaquery(query)

,first_name,last_name,league_minutes,league_yellow
0,Jordi,Alba,3030,9


Which defender contribute the most goals and assists in the league?

In [48]:
query = '''
        SELECT
        squad.first_name,
        squad.last_name,
        league_stats.league_goals + league_stats.league_assists AS goal_involvment
        FROM
        league_stats
        INNER JOIN
        squad ON squad.player_id = league_stats.player_id
        WHERE
        squad.position = 'df'
        ORDER BY
        goal_involvment DESC
        LIMIT 1;
        '''
Pandaquery(query)

,first_name,last_name,goal_involvment
0,Jordi,Alba,10


How many players played more then 50 hours total in the league and in the champions league?

In [50]:
query = '''
        SELECT
        COUNT(squad.player_id)
        FROM
        squad
        INNER JOIN league_stats ON squad.player_id = league_stats.player_id
        INNER JOIN champions_stats ON champions_stats.player_id = squad.player_id
        WHERE
        league_stats.league_minutes + champions_stats.champions_minutes > 3000
        ;
        '''
Pandaquery(query)

,count
0,6
